# [Module 4] Prophet 및 DeepAR+ Forecast 생성 (Create a Target Forecast)
- 이전 모듈에서 생성한 Predictor를 가지고 Prophet 및 DeepAR+ Forecast 생성을 합니다. Foreast의 의미는 실질적으로 전체 학습을 다시하고 서비스를 할 수 있는 준비를 하는 과정 입니다. Forecast의 생성 이후에 실제 미래의 예측(Inference)을 일괄적으로 하여 S3에 저장을 합니다.

* **About 50 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables stored by other notebooks
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create Prophet and DeepAR+ Forecast

In [4]:
# Prophet
prophet_forecastName = project+'_prophet_algo_forecast' + target_suffix + suffix
prophet_create_forecast_response=forecast.create_forecast(
    ForecastName=prophet_forecastName,
    PredictorArn=target_prophet_predictorArn)
target_prophet_forecast_arn = prophet_create_forecast_response['ForecastArn']

In [5]:
forecast.describe_forecast(ForecastArn = target_prophet_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:ap-northeast-2:728311679782:forecast/StoreItemDemand_prophet_algo_forecast_target39275',
 'ForecastName': 'StoreItemDemand_prophet_algo_forecast_target39275',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:ap-northeast-2:728311679782:predictor/StoreItemDemand_prophet_algo_1_target39275',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:728311679782:dataset-group/StoreItemDemandDSG39275',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 2, 7, 3, 32, 890000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 2, 7, 3, 32, 890000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c583308d-f1c8-4380-ad5f-ff6fc93daf63',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 02 Jul 2020 07:03:32 GMT',
   'x-amzn-requestid': 'c583308d-f1c8-4380-ad5f-ff6fc93daf63',
   'content-length': '538',
   'connection': 'keep-alive'},
  'Ret

In [6]:
# DeepAR+
deeparp_forecastName = project+'_deeparp_algo_forecast' + target_suffix + suffix
deeparp_create_forecast_response=forecast.create_forecast(
    ForecastName=deeparp_forecastName,
    PredictorArn=target_deepar_predictorArn)
target_deeparp_forecast_arn = deeparp_create_forecast_response['ForecastArn']

In [7]:
forecast.describe_forecast(ForecastArn = target_deeparp_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:ap-northeast-2:728311679782:forecast/StoreItemDemand_deeparp_algo_forecast_target39275',
 'ForecastName': 'StoreItemDemand_deeparp_algo_forecast_target39275',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:ap-northeast-2:728311679782:predictor/StoreItemDemand_deeparp_algo_139275',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:728311679782:dataset-group/StoreItemDemandDSG39275',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 2, 7, 3, 33, 401000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 2, 7, 3, 33, 401000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '27dde83d-2f0e-4fa2-bd6e-96a3c9a6a3d3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 02 Jul 2020 07:03:33 GMT',
   'x-amzn-requestid': '27dde83d-2f0e-4fa2-bd6e-96a3c9a6a3d3',
   'content-length': '533',
   'connection': 'keep-alive'},
  'RetryAttem

In [8]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast(ForecastArn= target_prophet_forecast_arn)['Status']
    createDeeparpStatus = forecast.describe_forecast(ForecastArn= target_deeparp_forecast_arn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PR

## Upload forecast results to S3
- Forecast를 가지고 일괄적으로 예측 작업을 하여 결과를 S3에 저장 합니다.

In [9]:
target_prophet_path = "s3://" + bucket_name + "/" + bucket_folder + "/prophet_" + target_suffix + suffix + "/" 
target_prophet_job_name = "ProphetExport1" + target_suffix + suffix
create_forecast_export_job_prophet_response = forecast.create_forecast_export_job(
    ForecastExportJobName = target_prophet_job_name,
    ForecastArn = target_prophet_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": target_prophet_path,
            "RoleArn": role_arn
        }
    })


In [10]:
TargetForecastProphetExportJobArn = create_forecast_export_job_prophet_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = TargetForecastProphetExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:ap-northeast-2:728311679782:forecast-export-job/StoreItemDemand_prophet_algo_forecast_target39275/ProphetExport1_target39275',
 'ForecastExportJobName': 'ProphetExport1_target39275',
 'ForecastArn': 'arn:aws:forecast:ap-northeast-2:728311679782:forecast/StoreItemDemand_prophet_algo_forecast_target39275',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-ap-northeast-2-728311679782/StoreItemDemand/prophet__target39275',
   'RoleArn': 'arn:aws:iam::728311679782:role/ForecastRolePOC39275'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 2, 7, 38, 40, 816000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 2, 7, 38, 40, 816000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '773228d1-f31e-4a93-ba1e-d092f0f7bf93',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 02 Jul 2020 07:38:40 GMT',
   'x-amzn-requestid': '773228d1-f31e-4a93-b

In [11]:
target_deeparp_path = "s3://" + bucket_name + "/" +  bucket_folder +  "/deeparp_" + target_suffix + suffix + "/" 
target_deeparp_job_name = "DeepARPExport1" + target_suffix + suffix
create_forecast_export_job_deepar_response = forecast.create_forecast_export_job(
    ForecastExportJobName = target_deeparp_job_name,
    ForecastArn = target_deeparp_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": target_deeparp_path,
            "RoleArn": role_arn
        }
    })

In [12]:
TargetForecastDeeparExportJobArn = create_forecast_export_job_deepar_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = TargetForecastDeeparExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:ap-northeast-2:728311679782:forecast-export-job/StoreItemDemand_deeparp_algo_forecast_target39275/DeepARPExport1_target39275',
 'ForecastExportJobName': 'DeepARPExport1_target39275',
 'ForecastArn': 'arn:aws:forecast:ap-northeast-2:728311679782:forecast/StoreItemDemand_deeparp_algo_forecast_target39275',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-ap-northeast-2-728311679782/StoreItemDemand/deeparp__target39275',
   'RoleArn': 'arn:aws:iam::728311679782:role/ForecastRolePOC39275'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 2, 7, 38, 41, 123000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 2, 7, 38, 41, 123000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'f79db370-9c99-486d-9455-df105637aee6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 02 Jul 2020 07:38:41 GMT',
   'x-amzn-requestid': 'f79db370-9c99-486d-9

This exporting process is another one of those items that will take several minutes to complete. Just poll for progress in the console. From the earlier page where you saw the status turn `Active` for a Forecast, click it and you can see the progress of the export.

In [13]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= TargetForecastProphetExportJobArn)['Status']
    createDeeparpStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= TargetForecastDeeparExportJobArn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 88 ms, sys: 19.1 ms, total: 107 ms
Wall time: 8min 1s


In [14]:
%store target_prophet_forecast_arn
%store target_deeparp_forecast_arn
%store TargetForecastProphetExportJobArn
%store TargetForecastDeeparExportJobArn



Stored 'target_prophet_forecast_arn' (str)
Stored 'target_deeparp_forecast_arn' (str)
Stored 'TargetForecastProphetExportJobArn' (str)
Stored 'TargetForecastDeeparExportJobArn' (str)
